# Библиотеки

In [ ]:
import torch
from tqdm import tqdm
import torchvision
import random
import numpy as np
from IPython.display import clear_output

try:
    import transformers
except ModuleNotFoundError:
    !pip install transformers
    !pip install transformers sentencepiece --quiet
    clear_output()
    import transformers
from transformers import (
    T5ForConditionalGeneration, T5TokenizerFast, T5Tokenizer,
    DataCollatorForLanguageModeling, TrainingArguments, Trainer,
)

try:
    import datasets
except ModuleNotFoundError:
    !pip install datasets
    clear_output()
from datasets import load_dataset

try:
    import rouge
except ModuleNotFoundError:
    ! pip install rouge
    clear_output()
from rouge import Rouge

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

try:
    import nltk
except ModuleNotFoundError:
    ! pip install nltk
    clear_output()
from nltk.translate.bleu_score import corpus_bleu

try:
    import evaluate
except ModuleNotFoundError:
    ! pip install evaluate
    ! pip install bert_score
    clear_output()
from evaluate import load

Фиксируем сиды

In [ ]:
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)
#torch.use_deterministic_algorithms(True)

Пути для сохранения - измените и не забудьте подключить диск:

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks'
saved_model_path = path + '/archive'
checkpoint_path = path + '/t5-model-small'
logs_path = checkpoint_path + '/logs'
dataset_path = path + '/dataset'
model_name = "cointegrated/rut5-small"
pretrained_model = saved_model_path + '/model_t5_small_4.pth'
test_path = path + '/texts'

Устройство ускорителя:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка

Функции:

In [ ]:
def make_dataset(data, tokenizer, max_length_text=2890, max_length_ref=200):
    '''
    Создать датасет для обучения модели: исходный текс обрабатывается токенизатором, а затем к полученному словарю добавляется метка label
    с токенизированным эталонным рефератом

    Возвращает преобразованный датасет (list)
    '''
    dataset = []
    for inst in tqdm(data):
        txt = tokenizer(inst['text'], add_special_tokens=True, max_length=max_length_text, padding="max_length", truncation=True)
        sum_ = tokenizer(inst['summary'], add_special_tokens=True, max_length=max_length_ref, padding="max_length", truncation=True).input_ids
        txt["labels"] = sum_
        dataset.append(txt)
    return dataset

def save(path, model, optimizer):
    '''
    Сохранить модель и оптимизатор в path
    '''
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, path)

def summarize(text, model, tokenizer, max_length_text=2890, max_length_ref=500):
    '''
    Генерация реферата

    Возвращает сгенерированный моделью реферат (str)
    '''
    inp = tokenizer(text, add_special_tokens=True, max_length=max_length_text, padding="max_length", truncation=True, return_tensors='pt').to(device)
    return tokenizer.decode(model.generate(input_ids=inp.input_ids, attention_mask=inp.attention_mask, max_length=max_length_ref)[0], skip_special_tokens=True)

def tests_res(data, model, tokenizer, max_length_text=2890):
    '''
    Генерация рефератов для тестирования модели

    Возвращает результат модели на датасете data (list) и эталонные рефераты (list)
    '''
    res, ref = [], []
    for inst in tqdm(data):
        res.append(summarize(inst['text'], model, tokenizer))
        ref.append(inst['summary'])
    return res, ref

def read_dataset(model, tokenizer, max_length_text=2890, max_length_ref=200, n=50):
    '''
    Создать датасет для обучения (для чтения из директории)

    Возвращает преобразованный датасет (list)
    '''
    dataset = []
    for i in range(n):
        with open(dataset_path + f'/{i}.txt') as f:
            data = f.read()
        data = data.split('\n\n')
        txt = tokenizer(data[1], add_special_tokens=True, max_length=max_length_text, padding="max_length", truncation=True)
        sum_ = tokenizer(data[2], add_special_tokens=True, max_length=max_length_ref, padding="max_length", truncation=True).input_ids
        txt["labels"] = sum_
        dataset.append(txt)
    return dataset


Загрузка модели:

In [ ]:
# Загрузка модели
tokenizer = T5Tokenizer.from_pretrained(model_name)
model_t5 = T5ForConditionalGeneration.from_pretrained(model_name)
optimizer = torch.optim.AdamW(model_t5.parameters(),lr=1e-4)

checkpoint = torch.load(pretrained_model, map_location='cpu')
model_t5.load_state_dict(checkpoint['model_state_dict'])
model_t5.to(device)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Загрузка датасета Gazeta и создание обучающей + валидационной выборок

In [ ]:
# Загрузка датасета
dataset_train = load_dataset('IlyaGusev/gazeta', revision="v2.0")["train"]
dataset_val = load_dataset('IlyaGusev/gazeta', revision="v2.0")["validation"]
dataset_test = load_dataset('IlyaGusev/gazeta', revision="v2.0")["test"]
clear_output()

In [ ]:
# Создание обучающей и валидационной выборок
print('Making train dataset')
dataset_train = make_dataset(dataset_train, tokenizer)
print('Making val dataset')
dataset_val = make_dataset(dataset_val, tokenizer)
print('Done!')

Загрузка метрик

In [ ]:
rouge = Rouge()
meteor = load('meteor')
bertscore = load("bertscore")

# Main

Если возникла ошибка, запустите код и перезапустите ноутбук:

In [ ]:
'''
#if "Using the `Trainer` with `PyTorch` requires `accelerate`: Run `pip install --upgrade accelerate`" occured:
! pip uninstall -y transformers accelerate
! pip install transformers accelerate
clear_output()
# then reload notebook
'''

## Дообучение на датасете Gazeta:

Функция для одного прогона обучение+текстирование (с логированием)

In [ ]:
def train_test_model(model, tokenizer, optimizer,
                     test_dataset,
                     num_steps=1, ):

    for step in range(1, num_steps+1):
        # параметры для Тренера
        training_args = TrainingArguments(
            output_dir= checkpoint_path,
            overwrite_output_dir=True,
            per_device_train_batch_size=2,
            per_device_eval_batch_size=2,
            num_train_epochs=1,
            warmup_steps=10,
            gradient_accumulation_steps=16,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            seed=42,
        )

        # Тренер
        trainer = Trainer(
            model=model_t5,
            args=training_args,
            train_dataset=dataset_train,
            eval_dataset=dataset_val,
            tokenizer=tokenizer,
            optimizers = (optimizer, None)
        )

        # Обучение модели
        model.train()
        logs = trainer.train()
        print('Saving model')
        save(saved_model_path + f'/model_t5_small_5_{step}.pth', model, optimizer)

        print('Saving loss')
        with open(logs_path + f'/loss_dictionary.txt','a') as f:
            f.write(f'Step_{step}\nTRAIN LOG:\n{logs}\n')

        # Тестирование модели
        model.eval()
        print('Testing')
        model_results, refs = tests_res(test_dataset, model, tokenizer) # результаты работы модели

        # Оценка ROUGE
        print('Done! Counting Rouge')
        scores = rouge.get_scores(model_results, refs, avg=True)
        print(scores)

        # Оценка BLEU
        print('Done! Counting BLEU')
        blue = corpus_bleu([[r.split(" ")] for r in refs], [hyp.split(" ") for hyp in model_results])
        print(blue)

        # Оценка METEOR
        print('Done! Counting METEOR')
        results_m = meteor.compute(predictions=model_results, references=refs)
        print(results_m)

        # Оценка BertScore
        print('Done! Counting BertScore')
        results_b = bertscore.compute(predictions=model_results, references=refs, lang="ru")
        results_b = {k: np.mean(v) for k, v in list(results_b.items())[:-1]}
        print(results_b)

        print('Saving scores')
        with open(logs_path + f'/metrics.txt', 'a') as f:
            f.write(f'STEP: {step}\n')
            f.write(f'ROUGE: {scores}\n')
            f.write(f'BLEU: {blue}\n')
            f.write(f'METEOR: {results_m}\n')
            f.write(f'BertScore: {results_b}\n\n')

In [ ]:
num_steps = 1

train_test_model(model_t5, tokenizer, optimizer, dataset_test, num_steps=num_steps)

## Тестирование на своей выборке текстов:

In [ ]:
model_t5.eval()

In [ ]:
for i in range(64):
    t = open(test_path + f'/text_{i}', 'r', encoding='cp1251').read()
    print(f'{i}) {summarize(t, model_t5, tokenizer)}')
    print('-----------')

0) В работе рассматривается метод емкостной деионизации водного раствора, которые можно будет пропускать водного раствора через электрохимическую ячейку между двумя пористыми электродами. Это более энергоемкий метод опреснения воды в сравнении с методами обратного осмоса и дистилляции.
-----------
1) Для новых фактов и теорем можно будет использовать доказательные вычисления (ДВ) — вычисления, направленные на доказательство новых фактов и теорем и основанные на совместном применении численных и аналитических методов.
-----------
2) Для выделения гистологических изображений желез можно получить точные результаты для большого объема данных. Это было одним из основных критерий для успешного успешного успешного алгоритма сегментации гистологических изображений.
-----------
3) Методы подавления шума на изображениях можно будет работать в одном из основных задач в области обработки изображений. Это одна из основных задач в области обработки изображений.
-----------
4) Метод автоматического а

## Дообучение на своем датасете:

Загрузка датасета и создание выборки

In [ ]:
train_set = read_dataset(model_t5, tokenizer)

test_set = []
for i in range(50, 70):
    with open(dataset_path + f'/{i}.txt') as f:
        data = f.read()
        data = data.split('\n\n')
        test_set.append({'text': data[1], 'summary': data[2]})

Тренер:

In [ ]:
model_t5.train()

# параметры для Тренера
training_args = TrainingArguments(
    output_dir= checkpoint_path,
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=1,
    warmup_steps=10,
    gradient_accumulation_steps=16,
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=42,
)

# Тренер
trainer = Trainer(
    model=model_t5,
    args=training_args,
    train_dataset=train_set,
    tokenizer=tokenizer,
    optimizers = (optimizer, None)
)

Обучение + тестирование:

In [ ]:
num_steps = 1

train_test_model(model_t5, tokenizer, optimizer, trainer, test_set, num_steps=num_steps)

## Тестирование на своей выборке текстов:

In [ ]:
model_t5.eval()

In [ ]:
for i in range(64):
    t = open(test_path + f'/text_{i}', 'r', encoding='cp1251').read()
    print(f'{i}) {summarize(t)}')
    print('-----------')